In [1]:
import os
import shutil
import pandas as pd
import pydicom
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import cv2
import numpy as np

In [2]:
# Path to the Excel file containing the MRI coordinates
targeted_bx_path = '/Users/arjunmoorthy/Desktop/Research_Capstone/targeted_biopsy_patients.xlsx'

# Directory containing the DICOM images
dicom_dir_path = '/Users/arjunmoorthy/Desktop/targeted_bx_dicoms'

# Load the Excel file
targeted_df = pd.read_excel(targeted_bx_path)

In [3]:
patient_id = 'Prostate-MRI-US-Biopsy-0003'  # Test Patient ID

# Path to the patient's DICOM directory
patient_dicom_dir = os.path.join(dicom_dir_path, patient_id)
print(f"Searching in: {patient_dicom_dir}")  

z_coords = []
output_dir_path = os.path.join(dicom_dir_path, 'targeted_bx_slices_bothz')
# Walk through all subdirectories in the patient's DICOM directory
for root, dirs, files in os.walk(patient_dicom_dir):
    for file in files:
        if file.endswith(".dcm"):
            file_path = os.path.join(root, file)
            print(f"Reading DICOM: {file_path}")  
            ds = pydicom.dcmread(file_path)

            # Extract the z-coordinate
            if 'ImagePositionPatient' in ds:
                dicom_z_coord = ds.ImagePositionPatient[2]
                z_coords.append(round(dicom_z_coord, 2)) 
                print(f"Found z-coordinate: {dicom_z_coord}")  

# Check if any z-coordinates were found and print them
if z_coords:
    print(f"Z-coordinates for patient {patient_id}: {z_coords}")
else:
    print("No z-coordinates found.")


Searching in: /Users/arjunmoorthy/Desktop/targeted_bx_dicoms/Prostate-MRI-US-Biopsy-0003
Reading DICOM: /Users/arjunmoorthy/Desktop/targeted_bx_dicoms/Prostate-MRI-US-Biopsy-0003/03-22-2009-NA-MRI PROSTATE W WO CONTRAST-74733/13.000000-t2spcrstaxial oblProstate-44248/1-42.dcm
Found z-coordinate: 83.257481113076
Reading DICOM: /Users/arjunmoorthy/Desktop/targeted_bx_dicoms/Prostate-MRI-US-Biopsy-0003/03-22-2009-NA-MRI PROSTATE W WO CONTRAST-74733/13.000000-t2spcrstaxial oblProstate-44248/1-56.dcm
Found z-coordinate: 104.25748111308
Reading DICOM: /Users/arjunmoorthy/Desktop/targeted_bx_dicoms/Prostate-MRI-US-Biopsy-0003/03-22-2009-NA-MRI PROSTATE W WO CONTRAST-74733/13.000000-t2spcrstaxial oblProstate-44248/1-57.dcm
Found z-coordinate: 105.75748111308
Reading DICOM: /Users/arjunmoorthy/Desktop/targeted_bx_dicoms/Prostate-MRI-US-Biopsy-0003/03-22-2009-NA-MRI PROSTATE W WO CONTRAST-74733/13.000000-t2spcrstaxial oblProstate-44248/1-43.dcm
Found z-coordinate: 84.757481113076
Reading DICOM: 

In [4]:
# Assuming dicom_dir_path and targeted_bx_path are defined
output_dir_path = os.path.join(dicom_dir_path, 'targeted_bx_slices_bothz')

# Ensure the output directory exists
os.makedirs(output_dir_path, exist_ok=True)

# Load the Excel file
df = pd.read_excel(targeted_bx_path)

# Dictionary of Dicom data
closest_slices = {}  # Format: {z_coord: (file_path, [list of (x,y) coords], min_diff)}

def find_and_copy_closest_slices(patient_id, series_id, bx_points):
    patient_dicom_dir = os.path.join(dicom_dir_path, patient_id)
    # closest_slices = {}  # Format: {z_coord: (file_path, [list of (x,y) coords], min_diff)}

    # Walk through all subdirectories of the patient's DICOM directory
    for root, dirs, files in os.walk(patient_dicom_dir):
        for file in files:
            if file.endswith(".dcm"):
                file_path = os.path.join(root, file)
                ds = pydicom.dcmread(file_path)
                if ds.SeriesInstanceUID == series_id:
                    if 'ImagePositionPatient' in ds:
                        dicom_z_coord = ds.ImagePositionPatient[2]
                        for z_coord, xy_coords in bx_points.items():
                            diff = abs(dicom_z_coord - z_coord)

                            if z_coord not in closest_slices or diff < closest_slices[z_coord][2]:
                                closest_slices[z_coord] = (file_path, [xy_coords], diff)
                            elif diff == closest_slices[z_coord][2]:
                                closest_slices[z_coord][1].append(xy_coords)  # Add xy_coords if the diff is equal
    
    # Copy the closest slices to the output directory
    output_patient_dir = os.path.join(output_dir_path, patient_id)
    os.makedirs(output_patient_dir, exist_ok=True)
    
    for z_coord, (file_path, xy_coords_list, _) in closest_slices.items():
        shutil.copy(file_path, output_patient_dir)
        print(f"Copied: {file_path} to {output_patient_dir} for z={z_coord} with xy points {xy_coords_list}")
    
    return closest_slices

targeted_patient_xy_info = []

# Convert DataFrame rows into a suitable structure for processing
for index, row in df.iterrows():
    patient_id = str(row['Patient Number'])
    series_id = str(row['Series Instance UID (MRI)'])
    if patient_id=='Prostate-MRI-US-Biopsy-0005': break
    bx_points = {
        row['Bx Tip Z (MRI Coord)']: (row['Bx Tip X (MRI Coord)'], row['Bx Tip Y (MRI Coord)']),
        row['Bx Base Z (MRI Coord)']: (row['Bx Base X (MRI Coord)'], row['Bx Base Y (MRI Coord)'])
    }
    data = find_and_copy_closest_slices(patient_id, series_id, bx_points)

    targeted_patient_xy_info.append(data)




Copied: /Users/arjunmoorthy/Desktop/targeted_bx_dicoms/Prostate-MRI-US-Biopsy-0001/06-28-2009-NA-MRI PROSTATE W WO CONTRAST-51743/11.000000-t2spcrstaxial oblProstate-90221/1-23.dcm to /Users/arjunmoorthy/Desktop/targeted_bx_dicoms/targeted_bx_slices_bothz/Prostate-MRI-US-Biopsy-0001 for z=-41.906 with xy points [(-28.54, 15.165)]
Copied: /Users/arjunmoorthy/Desktop/targeted_bx_dicoms/Prostate-MRI-US-Biopsy-0001/06-28-2009-NA-MRI PROSTATE W WO CONTRAST-51743/11.000000-t2spcrstaxial oblProstate-90221/1-15.dcm to /Users/arjunmoorthy/Desktop/targeted_bx_dicoms/targeted_bx_slices_bothz/Prostate-MRI-US-Biopsy-0001 for z=-53.173 with xy points [(-35.082, 34.427)]
Copied: /Users/arjunmoorthy/Desktop/targeted_bx_dicoms/Prostate-MRI-US-Biopsy-0001/06-28-2009-NA-MRI PROSTATE W WO CONTRAST-51743/11.000000-t2spcrstaxial oblProstate-90221/1-23.dcm to /Users/arjunmoorthy/Desktop/targeted_bx_dicoms/targeted_bx_slices_bothz/Prostate-MRI-US-Biopsy-0001 for z=-41.906 with xy points [(-28.54, 15.165)]
Cop

In [5]:
# Folder for segmentations
seg_folder = '/Users/arjunmoorthy/Desktop/Research_Capstone/Image Data/Prostate-MRI-US-Biopsy'

In [10]:
data[0]

{-41.906: ('/Users/arjunmoorthy/Desktop/targeted_bx_dicoms/Prostate-MRI-US-Biopsy-0001/06-28-2009-NA-MRI PROSTATE W WO CONTRAST-51743/11.000000-t2spcrstaxial oblProstate-90221/1-23.dcm',
  [(-28.54, 15.165)],
  0.0249259886739992),
 -53.173: ('/Users/arjunmoorthy/Desktop/targeted_bx_dicoms/Prostate-MRI-US-Biopsy-0001/06-28-2009-NA-MRI PROSTATE W WO CONTRAST-51743/11.000000-t2spcrstaxial oblProstate-90221/1-15.dcm',
  [(-35.082, 34.427)],
  0.7080740113259978),
 -40.452: ('/Users/arjunmoorthy/Desktop/targeted_bx_dicoms/Prostate-MRI-US-Biopsy-0001/06-28-2009-NA-MRI PROSTATE W WO CONTRAST-51743/11.000000-t2spcrstaxial oblProstate-90221/1-24.dcm',
  [(-32.174, 24.978)],
  0.07092598867399857),
 -52.446: ('/Users/arjunmoorthy/Desktop/targeted_bx_dicoms/Prostate-MRI-US-Biopsy-0001/06-28-2009-NA-MRI PROSTATE W WO CONTRAST-51743/11.000000-t2spcrstaxial oblProstate-90221/1-16.dcm',
  [(-38.716, 40.969)],
  0.06492598867399835),
 -46.631: ('/Users/arjunmoorthy/Desktop/targeted_bx_dicoms/Prostate

In [11]:
data_new =  {}
for z_coord,data in targeted_patient_xy_info[0].items():
    # print(z_coord,data)
    img_path = data[0]
    patient_name = data[0].split('/')[5]
    img = pydicom.dcmread(img_path)
    imagePositionPatient = img.ImagePositionPatient
    study_id = img.StudyInstanceUID
    PixelSpacing = img.PixelSpacing
    instance_number = int(img.InstanceNumber)
    lps_x,lps_y = data[1][0]
    voxel_cord_x = round((lps_x-imagePositionPatient[0])/PixelSpacing[0])
    voxel_cord_y = round((lps_y-imagePositionPatient[1])/PixelSpacing[1])
    voxel_cord = [voxel_cord_x,voxel_cord_y,instance_number]
    
    tag_data = f'{patient_name}_{instance_number}'
    
    dicom_seg_path = f'{seg_folder}/{patient_name}/{study_id}'
    lesion_path = list(filter(lambda x:('lesion' in x) & ('surface' not in x),os.listdir(dicom_seg_path)))[0]
    lesion_image_path = f'{dicom_seg_path}/{lesion_path}'
    prostate_path = list(filter(lambda x:(x.endswith('prostate.dcm')) & ('surface' not in x),os.listdir(dicom_seg_path)))[0]               
    prostate_image_path = f'{dicom_seg_path}/{prostate_path}'
    
    prostate = pydicom.dcmread(prostate_image_path)
    prostate_arr = prostate.pixel_array
    prostate_arr = np.transpose(prostate_arr,(1,2,0)) #transpose to bring the axial axis to last position

    lesion = pydicom.dcmread(lesion_image_path)
    lesion_arr = lesion.pixel_array
    lesion_arr = np.transpose(lesion_arr,(1,2,0)) #transpose to bring the axial axis to last position
    
    seg = prostate_arr[:,:,instance_number-1]
    les = lesion_arr[:,:,instance_number-1]
    
    arr = img.pixel_array
    arr = np.stack((arr,) * 3, axis=-1)
    contours, _ = cv2.findContours(seg, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) #function to extract the contours from the given mask file
    contours2, _ = cv2.findContours(les, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) #function to extract the contours from the given mask file

    arr = cv2.drawContours(arr, contours, -1, (0, 255, 0), 2) #overlay the contour on the image
    arr = cv2.drawContours(arr, contours2, -1, (0, 0, 255), 2) #overlay the contour on the image

    add_data = {'path':img_path,'voxel_coords':voxel_cord,'instance_number':instance_number,'arr_img':arr}
    
    if tag_data not in data_new:
        data_new[tag_data] = [add_data]
    else:
        data_new[tag_data].append(add_data)
        

In [12]:
data_new

{'Prostate-MRI-US-Biopsy-0001_23': [{'path': '/Users/arjunmoorthy/Desktop/targeted_bx_dicoms/Prostate-MRI-US-Biopsy-0001/06-28-2009-NA-MRI PROSTATE W WO CONTRAST-51743/11.000000-t2spcrstaxial oblProstate-90221/1-23.dcm',
   'voxel_coords': [120, 122, 23],
   'instance_number': 23,
   'arr_img': array([[[ 0,  0,  0],
           [ 0,  0,  0],
           [ 0,  0,  0],
           ...,
           [ 0,  0,  0],
           [ 0,  0,  0],
           [ 0,  0,  0]],
   
          [[ 0,  0,  0],
           [ 0,  0,  0],
           [ 0,  0,  0],
           ...,
           [ 0,  0,  0],
           [ 1,  1,  1],
           [ 1,  1,  1]],
   
          [[ 0,  0,  0],
           [ 0,  0,  0],
           [ 0,  0,  0],
           ...,
           [29, 29, 29],
           [25, 25, 25],
           [ 7,  7,  7]],
   
          ...,
   
          [[ 0,  0,  0],
           [ 0,  0,  0],
           [ 0,  0,  0],
           ...,
           [ 0,  0,  0],
           [ 0,  0,  0],
           [ 0,  0,  0]],
   
    

In [15]:
thickness = 1
font = cv2.FONT_HERSHEY_SIMPLEX 
fontScale = 0.3
# Blue color in BGR 
color2 = (255, 128, 0)
op_dir = '/Users/arjunmoorthy/Desktop/annotated_imgs'
for idx,(key,value) in enumerate(data_new.items()):
    print(key)
    patient_name = key.rsplit('_')[0]
    op_folder = os.path.join(op_dir,patient_name)
    os.makedirs(op_folder, exist_ok=True)
    arr_img = value[0]['arr_img']
    
    for i in range(len(value)):
        
        voxel_cord = value[i]['voxel_coords']
        voxel_cord_x = voxel_cord[0]
        voxel_cord_y = voxel_cord[1]
        #extract the coords of x and y and plot it on the processed image as a circle point
        arr_img = cv2.circle(arr_img, (voxel_cord_x,voxel_cord_y), radius=1, color=color2, thickness=-1)
       # arr_img = cv2.putText(arr_img, text, (voxel_cord_x,voxel_cord_y-5), font,fontScale, color2, thickness, cv2.LINE_AA)
        
    cv2.imwrite(f'{op_folder}/{key}.jpg',cv2.cvtColor(arr_img,cv2.COLOR_BGR2RGB))

Prostate-MRI-US-Biopsy-0001_23
Prostate-MRI-US-Biopsy-0001_15
Prostate-MRI-US-Biopsy-0001_24
Prostate-MRI-US-Biopsy-0001_16
Prostate-MRI-US-Biopsy-0001_20
Prostate-MRI-US-Biopsy-0001_13
Prostate-MRI-US-Biopsy-0001_14
Prostate-MRI-US-Biopsy-0001_35
Prostate-MRI-US-Biopsy-0001_27
Prostate-MRI-US-Biopsy-0001_31
Prostate-MRI-US-Biopsy-0001_22
Prostate-MRI-US-Biopsy-0002_36
Prostate-MRI-US-Biopsy-0002_30
Prostate-MRI-US-Biopsy-0002_35
Prostate-MRI-US-Biopsy-0002_29
Prostate-MRI-US-Biopsy-0002_37
Prostate-MRI-US-Biopsy-0002_31
Prostate-MRI-US-Biopsy-0002_43
Prostate-MRI-US-Biopsy-0002_21
Prostate-MRI-US-Biopsy-0002_26
Prostate-MRI-US-Biopsy-0002_34
Prostate-MRI-US-Biopsy-0002_27
Prostate-MRI-US-Biopsy-0002_25
Prostate-MRI-US-Biopsy-0003_41
Prostate-MRI-US-Biopsy-0003_32
Prostate-MRI-US-Biopsy-0003_43
Prostate-MRI-US-Biopsy-0003_33
Prostate-MRI-US-Biopsy-0003_40
Prostate-MRI-US-Biopsy-0003_31
